# Superstore Business Intelligence Project  
## Notebook 01: Data Preparation & Feature Engineering  

### Objective
Prepare the SQL-cleaned Superstore dataset for analytical workflows by:

- Verifying structural integrity  
- Validating schema consistency  
- Performing essential date conversion  
- Creating business-relevant analytical features  
- Exporting an enriched dataset for further analysis  

This notebook establishes the technical foundation for EDA, statistical inference, and modeling.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings for better readability
pd.set_option('display.max_columns', None)

### Why This Step?

We apply display settings to:
- Avoid column truncation
- Maintain consistent decimal precision
- Improve readability for reporting

In [2]:
df = pd.read_csv('../data/superstore_clean_data.csv')

print("Dataset Shape:", df.shape)
df.head()

Dataset Shape: (9994, 21)


,Row_ID,Order_ID,Order_Date,Ship_Date,Ship_Mode,Customer_ID,Customer_Name,Segment,Country,City,State,Postal_Code,Region,Product_ID,Category,Sub_Category,Product_Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,Hon Deluxe Fabric Upholstered Stacking Chairs ...,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


### Dataset Source

The dataset used here is exported from MySQL after cleaning and schema enforcement.  
This ensures consistency between SQL analysis and Python workflow.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row_ID         9994 non-null   int64  
 1   Order_ID       9994 non-null   object 
 2   Order_Date     9994 non-null   object 
 3   Ship_Date      9994 non-null   object 
 4   Ship_Mode      9994 non-null   object 
 5   Customer_ID    9994 non-null   object 
 6   Customer_Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal_Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product_ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub_Category   9994 non-null   object 
 16  Product_Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

### Why Schema Verification?

Before performing analysis, we verify that data types align with the database schema.  
Incorrect data types can lead to calculation errors in later stages.

## Schema Verification

Observations:

- Numeric columns (Row_ID, Postal_Code, Quantity) are stored as integers.
- Financial variables (Sales, Profit, Discount) are stored as float.
- Categorical variables are stored as object type.
- Date columns are currently in object format and require conversion.

Schema aligns with SQL database structure.

In [4]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'])
df['Ship_Date'] = pd.to_datetime(df['Ship_Date'])

df[['Order_Date', 'Ship_Date']].dtypes

Order_Date    datetime64[ns]
Ship_Date     datetime64[ns]
dtype: object

### Why Convert Dates?

CSV imports load date columns as object type.  
Converting them to datetime format enables time-based calculations such as shipping duration and trend analysis.

In [5]:
print("Missing Values:\n")
print(df.isnull().sum())

print("\nDuplicate Rows:", df.duplicated().sum())

Missing Values:

Row_ID           0
Order_ID         0
Order_Date       0
Ship_Date        0
Ship_Mode        0
Customer_ID      0
Customer_Name    0
Segment          0
Country          0
City             0
State            0
Postal_Code      0
Region           0
Product_ID       0
Category         0
Sub_Category     0
Product_Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

Duplicate Rows: 0


### Data Integrity Check

We verify that:

- No missing values exist
- No duplicate records are present

Clean data ensures reliable downstream modeling.

## Data Integrity Summary

- No missing values detected.
- No duplicate records identified.
- Dataset is structurally clean and consistent.
- Date fields successfully converted to datetime format.

The dataset is technically ready for feature engineering.

_________________________________________________
## Feature Engineering Section
_________________________________________________

### Shipping Efficiency Features

Operational performance is measured using delivery duration and delay flag.

In [6]:
df['Shipping_Days'] = (df['Ship_Date'] - df['Order_Date']).dt.days
df['Late_Shipments'] = np.where(df['Shipping_Days'] > 5, 1, 0)

df[['Shipping_Days', 'Late_Shipments']].describe()

,Shipping_Days,Late_Shipments
count,9994.000000,9994.000000
mean,3.958175,0.182510
std,1.747567,0.386283
min,0.000000,0.000000
25%,3.000000,0.000000
50%,4.000000,0.000000
75%,5.000000,0.000000
max,7.000000,1.000000


In [7]:
df['Profit_Margin'] = df['Profit'] / df['Sales']
df['Loss_Flag'] = np.where(df['Profit'] < 0, 1, 0)

df[['Profit_Margin','Loss_Flag']].describe()

,Profit_Margin,Loss_Flag
count,9994.000000,9994.000000
mean,0.120314,0.187212
std,0.466754,0.390101
min,-2.750000,0.000000
25%,0.075000,0.000000
50%,0.270000,0.000000
75%,0.362500,0.000000
max,0.500000,1.000000


In [8]:
df['Discount_Bucket'] = pd.cut(
    df['Discount'],
    bins=[-0.01, 0, 0.2, 0.4, 0.6, 1],
    labels=['No Discount','Low','Moderate','High','Very High']
)

df[['Discount','Discount_Bucket']].head()

,Discount,Discount_Bucket
0,0.00,No Discount
1,0.00,No Discount
2,0.00,No Discount
3,0.45,High
4,0.20,Low


In [9]:
df['Order_Year'] = df['Order_Date'].dt.year
df['Order_Month'] = df['Order_Date'].dt.month
df['Order_Month_Name'] = df['Order_Date'].dt.month_name()

df[['Order_Year','Order_Month_Name']].head()

,Order_Year,Order_Month_Name
0,2016,November
1,2016,November
2,2016,June
3,2015,October
4,2015,October


## Feature Engineering Insights

The dataset has been enhanced with new analytical variables:

- Shipping_Days → Measures operational efficiency  
- Late_Shipment → Identifies delayed deliveries  
- Profit_Margin → Evaluates profitability relative to sales  
- Loss_Flag → Detects negative-profit transactions  
- Discount_Bucket → Categorizes discount intensity  
- Time Features → Enables trend and seasonality analysis  

These variables expand the dataset from transactional data to business intelligence data.

In [10]:
df.to_csv("../data/superstore_enriched.csv", index=False)
print("Enriched dataset exported successfully.")

Enriched dataset exported successfully.


## Notebook 01 Summary

This notebook transformed the SQL-cleaned Superstore dataset into an analysis-ready enriched dataset.

Key outcomes:

- Structural validation completed
- Data integrity verified
- Date columns converted
- Operational and profitability features engineered
- Enriched dataset exported as: superstore_enriched.csv

The dataset is now ready for:

Notebook 02 → Exploratory Data Analysis & Validation  
Notebook 03 → Statistical Inference  
Notebook 04 → Regression Modeling  
Notebook 05 → Distribution Modeling